In [1]:
# Import the schemas
from netket_schemas import LatticeSchema, HilbertSpaceSchema, HamiltonianSchema

## Supported Lattice Types

| Lattice Type   | Example Text Input           | Extent Format      | Notes                        |
|----------------|-----------------------------|--------------------|------------------------------|
| chain          | `chain of 8 sites`          | `[8]`              | 1D chain, length=int         |
| square         | `4x4 square lattice`        | `[4, 4]`           | Only n x n supported         |
| cube           | `2x2x2 cubic lattice`       | `[2, 2, 2]`        | Only n x n x n supported     |
| hypercube      | `hypercube 4`               | `[4]`              | Only single dimension        |
| triangular     | `3x3 triangular lattice`    | `[3, 3]`           | 2D, extent                   |
| kagome         | `kagome lattice 3x3`        | `[3, 3]`           | 2D, extent                   |
| honeycomb      | `honeycomb lattice 4x4`     | `[4, 4]`           | 2D, extent                   |
| fcc            | `2x2x2 fcc lattice`         | `[2, 2, 2]`        | 3D, extent                   |
| bcc            | `2x2x2 bcc lattice`         | `[2, 2, 2]`        | 3D, extent                   |
| pyrochlore     | `2x2x2 pyrochlore lattice`  | `[2, 2, 2]`        | 3D, extent                   |

---

## Supported Hilbert Space Types

| Space Type | Example Text Inputs | Required Fields | Constraints & Notes |
|------------|-------------------|-----------------|-------------------|
| spin       | - `spin-1/2 on each site`<br>- `spin-1 on each site` | `spin` | - Default is spin-1/2<br>- Defines spin space on each lattice site<br>- Supports arbitrary spin values |
| fermion    | - `4 fermions with spin-1/2`<br>- `2 fermions`<br>- `3 spin-1/2 fermions in 4 sites` | `n_particles`, `spin` | - Follows Pauli exclusion principle<br>- Maximum filling depends on spin and sites<br>- Spin-1/2 is most common case |
| boson      | - `3 bosons`<br>- `2 bosons in 3 modes`<br>- `5 bosons per site` | `n_particles`, `n_modes` (optional) | - No particle number restrictions<br>- Modes default to 1 if not specified<br>- Can specify local/global constraints |

---

## Supported Hamiltonian Interactions

| Model Type         | Example Text Input                        | Supported Parameters         | Hamiltonian Equation                                                                                  |
|--------------------|-------------------------------------------|-----------------------------|------------------------------------------------------------------------------------------------------|
|   Hubbard          | `Hubbard model with t=1, U=4`             | `t`, `U`                    | $ H = -t \sum_{\langle i,j \rangle, \sigma} (c_{i\sigma}^\dagger c_{j\sigma} + h.c.) + U \sum_i n_{i\uparrow} n_{i\downarrow} $ |
|   Extended Hubbard | `Extended Hubbard: t=1, U=4, V=0.5`     | `t`, `U`, `V`               | $ H = -t \sum_{\langle i,j \rangle, \sigma} (c_{i\sigma}^\dagger c_{j\sigma} + h.c.) + U \sum_i n_{i\uparrow} n_{i\downarrow} + V \sum_{\langle i,j \rangle} n_i n_j $ |
|   Heisenberg       | `Heisenberg model with J=1, hx=0.5`       | `J`, `hx`, `hz`             | $ H = J \sum_{\langle i,j \rangle} \vec{S}_i \cdot \vec{S}_j + h_x \sum_i S_i^x + h_z \sum_i S_i^z $ |
|   Ising            | `Ising model with Jz=1, hx=0.2`           | `Jz`, `hx`, `hz`            | $ H = J_z \sum_{\langle i,j \rangle} S_i^z S_j^z + h_x \sum_i S_i^x + h_z \sum_i S_i^z $           |
|   Fermion hopping  | `Fermion hopping t=1 and Zeeman field B=0.1` | `t`, `B`                | $ H = -t \sum_{\langle i,j \rangle, \sigma} (c_{i\sigma}^\dagger c_{j\sigma} + h.c.) + B \sum_i (n_{i\uparrow} - n_{i\downarrow}) $ |
|   Boson            | `Boson model with t=1, mu=0.5`            | `t`, `mu`                   | $ H = -t \sum_{\langle i,j \rangle} (b_i^\dagger b_j + h.c.) - \mu \sum_i n_i $                    |

---

**Legend:**
- $ c_{i\sigma}^\dagger, c_{i\sigma} $: Fermionic creation/annihilation operators at site $ i $ with spin $ \sigma $
- $ n_{i\sigma} = c_{i\sigma}^\dagger c_{i\sigma} $: Number operator
- $ n_i = n_{i\uparrow} + n_{i\downarrow} $: Total number operator at site $ i $
- $ \vec{S}_i $: Spin operator at site $ i $
- $ S_i^x, S_i^z $: Spin-x and spin-z operators
- $ b_i^\dagger, b_i $: Bosonic creation/annihilation operators
- $ \langle i,j \rangle $: Sum over nearest neighbors
- $ h.c. $: Hermitian conjugate

---

**Notes:**
- All parameters (`t`, `U`, `V`, `J`, `Jz`, `hx`, `hz`, `B`, `mu`) are real numbers specified in the text input.
- All text inputs are case-insensitive and flexible in format.
- Only supported combinations will be parsed; invalid or unsupported text will raise a validation error.
- For custom models, extend the schema and parsing logic as needed.

In [2]:
print("=== Testing LatticeSchema ===\n")

test_cases = [
    # Valid cases
    ("chain of 8 sites", "chain", [8], True),
    ("4x4 square lattice", "square", [4, 4], True),
    ("2x2x2 cubic lattice", "cube", [2, 2, 2], True),
    ("hypercube 4", "hypercube", [4], True),
    ("3x3 triangular lattice", "triangular", [3, 3], True),
    ("kagome lattice 3x3", "kagome", [3, 3], True),
    ("honeycomb lattice 4x4", "honeycomb", [4, 4], True),
    ("2x2x2 fcc lattice", "fcc", [2, 2, 2], True),
    ("2x2x2 bcc lattice", "bcc", [2, 2, 2], True),
    ("2x2x2 pyrochlore lattice", "pyrochlore", [2, 2, 2], True),
    # Invalid cases
    ("4x5 square lattice", "square", [4, 5], False),
    ("2x2x3 cube lattice", "cube", [2, 2, 3], False),
    ("hypercube 4 5", "hypercube", [4, 5], False),
    ("chain of -1 sites", "chain", [-1], False),
    ("unknown lattice 3x3", "unknown", [3, 3], False),
]

header = f"{'Description':30} {'Type':12} {'Extent':15} {'Should Succeed':15} {'Result':8} {'Error'}"
print(header)
print('-' * len(header))

for desc, expected_type, expected_extent, should_succeed in test_cases:
    try:
        lattice = LatticeSchema(text=desc)
        # Only test schema parsing, not NetKet graph construction
        result = "✓"
        error = ""
    except Exception as e:
        result = "✗"
        error = str(e)
    print(f"{desc:30} {expected_type:12} {str(expected_extent):15} {str(should_succeed):15} {result:8} {error}")

=== Testing LatticeSchema ===

Description                    Type         Extent          Should Succeed  Result   Error
------------------------------------------------------------------------------------------
chain of 8 sites               chain        [8]             True            ✓        
4x4 square lattice             square       [4, 4]          True            ✓        
2x2x2 cubic lattice            cube         [2, 2, 2]       True            ✓        
hypercube 4                    hypercube    [4]             True            ✓        
3x3 triangular lattice         triangular   [3, 3]          True            ✓        
kagome lattice 3x3             kagome       [3, 3]          True            ✓        
honeycomb lattice 4x4          honeycomb    [4, 4]          True            ✓        
2x2x2 fcc lattice              fcc          [2, 2, 2]       True            ✓        
2x2x2 bcc lattice              bcc          [2, 2, 2]       True            ✓        
2x2x2 pyrochl

In [2]:
print("=== Testing HilbertSpaceSchema ===\n")

test_cases = [
    "spin-1/2 on each site",
    "4 fermions with spin-1/2",
    "3 bosons in 2 modes",
    "spin-1 on each site",
    "2 fermions",
    "5 bosons"
]

for text in test_cases:
    try:
        hilbert_space = HilbertSpaceSchema(text=text)
        print(f"✓ '{text}' → {hilbert_space.space_type}, spin={hilbert_space.spin}, n_particles={hilbert_space.n_particles}")
    except Exception as e:
        print(f"✗ '{text}' → ERROR: {e}")
    print()

=== Testing HilbertSpaceSchema ===

✓ 'spin-1/2 on each site' → spin, spin=0.5, n_particles=None

✓ '4 fermions with spin-1/2' → spin, spin=0.5, n_particles=None

✓ '3 bosons in 2 modes' → boson, spin=0.5, n_particles=3

✓ 'spin-1 on each site' → spin, spin=1.0, n_particles=None

✓ '2 fermions' → fermion, spin=0.5, n_particles=2

✓ '5 bosons' → boson, spin=0.5, n_particles=5



In [4]:
print("=== Testing HamiltonianSchema ===\n")

test_cases = [
    "Hubbard model with t=1, U=4",
    "Heisenberg model with J=1, hx=0.5",
    "Fermion hopping t=1 and Zeeman field B=0.1",
    "Extended Hubbard: t=1, U=4, V=0.5",
    "Ising model with Jz=1, hx=0.2"
]

for text in test_cases:
    try:
        hamiltonian = HamiltonianSchema(text=text)
        print(f"✓ '{text}' → {hamiltonian.model_type}")
        print(f"  Parameters: t={hamiltonian.hopping}, U={hamiltonian.hubbard_u}, J={hamiltonian.heisenberg_j}")
    except Exception as e:
        print(f"✗ '{text}' → ERROR: {e}")
    print()

=== Testing HamiltonianSchema ===

✓ 'Hubbard model with t=1, U=4' → hubbard
  Parameters: t=1.0, U=4.0, J=0.0

✓ 'Heisenberg model with J=1, hx=0.5' → heisenberg
  Parameters: t=0.0, U=0.0, J=1.0

✓ 'Fermion hopping t=1 and Zeeman field B=0.1' → hubbard
  Parameters: t=1.0, U=0.0, J=0.0

✓ 'Extended Hubbard: t=1, U=4, V=0.5' → hubbard
  Parameters: t=1.0, U=4.0, J=0.0

✓ 'Ising model with Jz=1, hx=0.2' → ising
  Parameters: t=0.0, U=0.0, J=0.0



In [3]:
print("=== Testing Complete System ===\n")

try:
    # Create components
    lattice = LatticeSchema(text="4x4 square lattice")
    hilbert_space = HilbertSpaceSchema(text="8 fermions with spin-1/2")
    hamiltonian = HamiltonianSchema(text="Hubbard model with t=1, U=4")
    
    print(f"Lattice: {lattice.text}")
    print(f"Hilbert Space: {hilbert_space.text}")
    print(f"Hamiltonian: {hamiltonian.text}")
    
    # Convert to NetKet objects
    graph = lattice.to_netket_graph()
    hilbert = hilbert_space.to_netket_hilbert(graph)
    
    print(f"NetKet Graph: {graph.n_nodes} nodes")
    print(f"NetKet Hilbert: {hilbert.n_states} states")
    
    # Test Hamiltonian conversion (requires fermion_builder)
    try:
        h_op = hamiltonian.to_netket_hamiltonian(hilbert, graph)
        print(f"NetKet Hamiltonian: {type(h_op).__name__}")
    except ImportError:
        print("Hamiltonian conversion requires fermion_builder module")
    
except Exception as e:
    print(f"Complete system test failed: {e}")

=== Testing Complete System ===

Lattice: 4x4 square lattice
Hilbert Space: spin-1/2 on each site
Hamiltonian: Hubbard model with t=1.0, U=4.0
NetKet Graph: 16 nodes
NetKet Hilbert: 65536 states
Complete system test failed: cannot specify sz for hilbert without spin property


In [4]:
print("=== Testing Edge Cases ===\n")

# Test invalid inputs
invalid_tests = [
    ("Invalid lattice", "invalid lattice description"),
    ("Empty hilbert space", ""),
    ("Invalid Hamiltonian", "invalid model with wrong parameters"),
]

for test_name, text in invalid_tests:
    try:
        if "lattice" in test_name:
            lattice = LatticeSchema(text=text)
        elif "hilbert" in test_name:
            hilbert_space = HilbertSpaceSchema(text=text)
        elif "Hamiltonian" in test_name:
            hamiltonian = HamiltonianSchema(text=text)
        print(f"✗ {test_name}: Should have failed but didn't")
    except Exception as e:
        print(f"✓ {test_name}: Correctly caught error - {e}")
    print()

=== Testing Edge Cases ===

✓ Invalid lattice: Correctly caught error - 1 validation error for LatticeSchema
  Value error, Could not parse lattice description: 'invalid lattice description'. Supported formats: '4x4 square lattice', 'chain of 8 sites', '2x2x2 cubic lattice', etc. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

✓ Empty hilbert space: Correctly caught error - 1 validation error for HilbertSpaceSchema
space_type
  Field required [type=missing, input_value={'text': ''}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

✓ Invalid Hamiltonian: Correctly caught error - 1 validation error for HamiltonianSchema
  Value error, Could not parse Hamiltonian description: 'invalid model with wrong parameters'. Supported formats: 'Hubbard model with t=1, U=4', 'Heisenberg model with J=1, hx=0.5', etc. [type=value_error, input_value={}, input_type=dict]
   

In [5]:
print("=== Testing Structured Input ===\n")

# Test creating schemas with structured data instead of text
try:
    # Structured lattice
    lattice = LatticeSchema(
        lattice_type="square",
        extent=[3, 3]
    )
    print(f"Structured lattice: {lattice.text}")
    
    # Structured particles
    particles = HilbertSpaceSchema(
        space_type="fermion",
        spin=0.5,
        n_particles=6
    )
    print(f"Structured particles: {particles.text}")
    
    # Structured Hamiltonian
    hamiltonian = HamiltonianSchema(
        hopping=1.0,
        hubbard_u=4.0,
        model_type="hubbard"
    )
    print(f"Structured Hamiltonian: {hamiltonian.text}")
    
except Exception as e:
    print(f"Structured input test failed: {e}")

=== Testing Structured Input ===

Structured lattice: 3x3 square lattice
Structured particles: 6 fermions with spin-1/2
Structured Hamiltonian: Hubbard model with t=1.0, U=4.0
